In [15]:
from opensearchpy import OpenSearch, RequestsHttpConnection

host = "example-endpoint.us-west-1.es.amazonaws.com"
region = "us-west-1"


username = "username"
password = "passwork"

auth = (username, password)

client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)


In [20]:
import json

with open("amendments.json", "r") as f:
    data = json.load(f)

In [21]:
data[0].keys()

dict_keys(['number', 'text', 'ratified', 'ratified_seconds', 'vector'])

In [16]:
# create a new index

index_body = {
    "settings": {"index.knn": True},
    "mappings": {
        "properties": {
            "vector": {"type": "knn_vector", "dimension": 384},
        },
    },
}

client.indices.create(index="amendments", body=index_body)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'amendments'}

In [22]:
# add documents to the index
for i, amendment in enumerate(data):
    client.index(
        index="amendments",
        body=amendment,
        id=i,
    )

In [23]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


In [45]:
index_name = "amendments"

q = "guns"
vector = model.encode(q).tolist()

query = {"from": 2, "size": 2, "query": {"knn": {"vector": {"vector": vector, "k": 2}}}}

response = client.search(body=query, index=index_name)


In [46]:
response["hits"]["hits"]

[{'_index': 'amendments',
  '_id': '4',
  '_score': 0.37659028,
  '_source': {'number': 5,
   'text': 'No person shall be held to answer for a capital, or otherwise infamous crime, unless on a presentment or indictment of a Grand Jury, except in cases arising in the land or naval forces, or in the Militia, when in actual service in time of War or public danger; nor shall any person be subject for the same offence to be twice put in jeopardy of life or limb; nor shall be compelled in any criminal case to be a witness against himself, nor be deprived of life, liberty, or property, without due process of law; nor shall private property be taken for public use, without just compensation.',
   'ratified': 'December 15, 1791',
   'ratified_seconds': -5618566804.0,
   'vector': [-0.05590737611055374,
    0.05880388244986534,
    -0.03206641227006912,
    -0.05126185342669487,
    0.012661373242735863,
    0.044595345854759216,
    0.1020422950387001,
    -0.07433958351612091,
    -0.027116702